-------
- # 요약
    - 기울기 소실
    - Relu
    - torch.optim
    - torch.nn.Sequential
    - Weight initialization
        - Xavier
            - Normal
            - Uniform
            - torch.nn.init.xavier_uniform_
        - He
            - Normal
            - Uniform
    - dropout
        - torch.nn.Dropout
    - batch Normalization
        - Internal Covariate Shift
        - torch.nn.BatchNorm1d
    
-------

## 코드

### 전처리


In [18]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [20]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [21]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [22]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

### 모델 정의



#### 단일 레이어


In [23]:
# MNIST data image of shape 28 * 28 = 784
linear = torch.nn.Linear(784, 10, bias=True).to(device)

In [24]:
# Initialization
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-1.2901,  0.3930, -0.1891,  ..., -0.1626, -0.3275,  0.5608],
        [-1.6757, -0.2454,  0.9221,  ...,  0.9559,  0.6160, -0.4226],
        [ 0.2444,  1.4810, -2.0906,  ...,  0.1206,  1.0044, -0.3737],
        ...,
        [ 0.6234,  1.8019, -2.7563,  ..., -0.5889, -0.5576,  0.7360],
        [-0.2871, -1.3313, -2.2248,  ...,  0.0309,  0.9180, -0.1482],
        [ 0.7678,  0.6624, -0.5362,  ...,  0.2338,  0.3688, -0.7182]],
       device='cuda:0', requires_grad=True)

In [25]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

#### 다층 레이어
- torch.nn.Sequential 을 활용해 여러 층의 레이어를 연결함


##### dropout
- 드랍아웃은 테스트 때가 아닌 훈련 떄에 적용해야 하기 때문에
- 학습 전에는 model.train()
- 테스트 때에는 model.eval() 
- 을 실행시켜 훈련중인지, 테스트중인지 선언해야한다


In [26]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()
drop_prob=0.2
dropout = torch.nn.Dropout(p=drop_prob)

In [27]:
# Initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.3702,  0.9602,  0.0058,  ...,  0.2505, -0.4255,  0.7078],
        [ 2.3142, -0.7832, -2.3232,  ...,  1.8333, -0.2913,  1.0314],
        [-1.1711, -0.0515, -0.2709,  ...,  0.2770,  0.1517,  0.0769],
        ...,
        [ 2.2172, -1.7196, -0.4555,  ..., -0.6781, -1.1871,  0.8935],
        [-0.5275, -0.5952,  0.1630,  ...,  0.0664, -0.0730, -0.7841],
        [-0.0081,  0.6073,  0.5671,  ..., -1.1104,  1.8268, -1.3351]],
       requires_grad=True)

In [28]:
# model
model = torch.nn.Sequential(linear1, relu,dropout, linear2, relu,dropout, linear3).to(device)

In [29]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### 훈련
- - criterion 사용하는 방법정도 체크

In [30]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 328.277801514
Epoch: 0002 cost = 85.737838745
Epoch: 0003 cost = 53.734146118
Epoch: 0004 cost = 38.244319916
Epoch: 0005 cost = 29.685379028
Epoch: 0006 cost = 23.627124786
Epoch: 0007 cost = 18.931381226
Epoch: 0008 cost = 15.957583427
Epoch: 0009 cost = 13.348909378
Epoch: 0010 cost = 11.480672836
Epoch: 0011 cost = 9.860776901
Epoch: 0012 cost = 8.409042358
Epoch: 0013 cost = 7.365782261
Epoch: 0014 cost = 6.271689415
Epoch: 0015 cost = 5.520791054
Learning finished


### 테스트

In [31]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.06369999796152115
Label:  8
Prediction:  3


/opt/conda/lib/python3.11/site-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/opt/conda/lib/python3.11/site-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
